## FastEmbed

In [1]:
from fastembed import TextEmbedding

In [2]:
embedding_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [3]:
query = "I just discovered the course. Can I join now?"

In [4]:
query_embedding = list(embedding_model.embed(query))
len(query_embedding[0])

512

In [5]:
query_embedding[0].min()

np.float64(-0.11726373885183883)

In [6]:
q_emb = query_embedding[0]

## Cosine similarity

In [7]:
import numpy as np
np.linalg.norm(query_embedding[0])

np.float64(1.0)

In [8]:
doc = 'Can I still join the course after the start date?'

In [9]:
doc_emb = list(embedding_model.embed(doc))[0]

In [10]:
q_emb.dot(doc_emb)

np.float64(0.9008528895674548)

## Ranking by cosine

In [11]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [12]:
text_embeddings = list(embedding_model.embed([doc['text'] for doc in documents]))

In [13]:
similarities = [(i, text_embedding.dot(q_emb)) for i, text_embedding in enumerate(text_embeddings)]
similarities.sort(key=lambda x: x[1], reverse=True)
similarities

[(1, np.float64(0.8182378156620136)),
 (2, np.float64(0.8085397445747489)),
 (0, np.float64(0.762968451872193)),
 (4, np.float64(0.7304499196411823)),
 (3, np.float64(0.7133078832064158))]

In [14]:
question_text_embeddings = list(embedding_model.embed([doc['question'] + ' ' + doc['text'] for doc in documents]))

In [15]:
similarities = [(i, question_text_embedding.dot(q_emb)) for i, question_text_embedding in enumerate(question_text_embeddings)]
similarities.sort(key=lambda x: x[1], reverse=True)
similarities

[(0, np.float64(0.851454319443226)),
 (1, np.float64(0.8436594005975433)),
 (2, np.float64(0.8408287224005012)),
 (4, np.float64(0.8086007917931166)),
 (3, np.float64(0.7755157657626952))]

## Choosing the embedding model

In [22]:
model_size = set()

for model in TextEmbedding.list_supported_models():
    model_size.add(model['dim'])

sorted(model_size)

[384, 512, 768, 1024]

In [36]:
for model in TextEmbedding.list_supported_models():
    if model['dim'] == 384:
        print(model['model'])

BAAI/bge-small-en
BAAI/bge-small-en-v1.5
snowflake/snowflake-arctic-embed-xs
snowflake/snowflake-arctic-embed-s
sentence-transformers/all-MiniLM-L6-v2
sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


In [24]:
model_name = 'BAAI/bge-small-en'

## Indexing with qdrant

In [23]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [25]:
from qdrant_client import QdrantClient, models

In [26]:
client = QdrantClient("http://localhost:6333")

In [27]:
collection_name = 'homework-2'

In [ ]:
if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=
            models.VectorParams(
                size=384,
                distance=models.Distance.COSINE,
            )
    )

    client.upsert(
        collection_name=collection_name,
        points=[
            models.PointStruct(
                id=idx,
                vector=models.Document(
                        text=doc['question'] + ' ' + doc['text'],
                        model=model_name,
                    ),
                payload={
                    "text": doc['text'],
                    "question": doc['question'],
                    "course": doc['course'],
                }
            )
            for idx, doc in enumerate(documents)
        ]
    )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

In [30]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_name 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [33]:
search(query).points

[ScoredPoint(id=14, version=0, score=0.87031734, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)]